# Tutorial 8a: Extracting data from images

*This tutorial was generated from a Jupyer notebook.  You can download the notebook [here](t8a_extracting_data_from_images.ipynb).*

In [ ]:
import glob
import os
import warnings

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.signal
import scipy.stats as st

# BE/Bi 103 utilities
import bebi103

# Image processing tools
import skimage
import skimage.io

# Import plotting tools
import matplotlib.pyplot as plt
import seaborn as sns

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables high res graphics inline (only use with static plots (non-Bokeh))
# SVG is preferred, but there is a bug in Jupyter with vertical lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

# Import Bokeh modules for interactive plotting
import bokeh.charts
import bokeh.io
import bokeh.plotting

# Display graphics in this notebook
bokeh.io.output_notebook()

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Before we can start working with images, we need to know how to do all of the more mundane things with them, like organizing, loading, storing, etc.  Images that you want to analyze come is all sorts of formats and are organized in many different ways.  One instrument may give you a TIFF stack consisting of 8-bit images.  Another may give you a directory with lots of images in it with different names.  And so many instruments give you data in some proprietary format.  You need to know how to deal with lots of file formats and organization.

In this course, we will analyze various images, including microscopy images.  I generally will give them to you in exactly the format I received them in.

In today's tutorial, we will use a data set from some one of the students in the class (Mike Abrams) and two class alumni (Claire Bedbrook and Ravi Nath) in the class to practice loading images, defining ROIs, and some basic skills for extracting information from images.

In this tutorial and throughout the class, we will use `scikit-image` to do almost all of our image processing.  It has continual development, with new features constantly being added.  For the fairly simple image processing we'll do in this course, it suffices for most of what we need.  The [course website](http://bebi103.caltech.edu/2015/links.html) has links to many other image processing toolkits, some Python-based and many not.  In particular, [OpenCV](http://opencv.org) is very powerful and has complete Python bindings, though not yet for the newest version 3.0.

## The data set

The data set comes from three Caltech graduate students, Mike Abrams, Claire Bedbrook, and Ravi Nath, who are working on studying day and night behavior in jellyfish.  They photograph jellyfish over time and study their pulsing behavior.  A typical image is shown below.

<img src="jellyfish.png">

A single jellyfish lives in each box.  The boxes are 4 by 4 inches.  The jellyfish constantly contract and relax in a pulsing motion.  Our task today is to get traces so we can measure pulse frequency.

We have two different data sets, day and night, each contained in their own directory.  Each image they acquired is stored as a TIFF image.  The frame rate was 15 frames per second.

Let's take a look at the structure of the directory with the images.  We will use the `glob` module, which allows us to specify string with wild cards to specify files of interest.

In [ ]:
# The directory containing daytime data
data_dir = '../data/abrams_et_al/Cassiopea_Pulsation/day'

# Glob string for images
im_glob = os.path.join(data_dir, '*.TIF')

# Get list of files in directory
im_list = glob.glob(im_glob)

# Let's look at the first 10 entries
im_list[:10]

We see that the images are nicely names so that successive frames are stored in order.  This is convenient, and when this is not the case, you should rename files so that when listed in alphabetical order, they also appear in temporal order.

## Loading and displaying images
The workhorse for much of our image processing is `scikit-image`.  We will load our first image using `scikit-image`.  When importing `scikit-image`, it is called `skimage`, and we need to import the submodules individually.  The submodule for reading and writing images is `skimage.io`.  We can use it to load and display an image.  Let's start with the first jellyfish image.

In [ ]:
# Read in the image using skimage
im = skimage.io.imread(im_list[0])

# Let's get information about the image
print(type(im), im.dtype, im.shape)

So, the image is stored as a NumPy array of 16-bit `int`s.  The array is 3$\times$480$\times$640, which means this is an RGB image.  The first index of the image indicates the channel (red, green, or blue).  Cameras often store black and white images as RGB, so let's see if all three channels are the same.

In [ ]:
# Test to see if each R, G, and B value is the same
((im[0,:,:] == im[1,:,:]) &  (im[1,:,:] == im[2:,:])).all()

Indeed they are, so we can just consider one of the channels in our analysis.

In [ ]:
# Just slice the red channel
im = im[0,:,:]

Now, let's use `skimage.io.imshow()` to take a look at the image.

In [ ]:
skimage.io.imshow(im)

To avoid the white gridlines (which we may or may not want), we can temporarily change Seaborn's axis style in a `with` block.

In [ ]:
with sns.axes_style('white'):
    skimage.io.imshow(im)

## Burning a scale bar

Remember, scientific images are not just pretty pictures. It is crucially important that you know the interpixel distance and that the images have a scale bar when displayed, at least for publication.

For some microscope set-ups, we already know the physical length corresponding to the interpixel distance.  We often take a picture of something with known dimension (such as a stage micrometer) to find the interpixel distance.  In the case of these jellyfish images, we know the boxes are 4$\times$4 inches.  We can get the locations of the boxes in units of pixels using the `plt.ginput()` function.  This function records the coordinates of mouse clicks on a figure.  We can't really do this with graphics in the Jupyter notebook, but using a figure window, I get the following results.

In [ ]:
xy = [(346.43976683937819, 246.4948186528498),
      (498.77137305699478, 251.93523316062181)]

We can use these points to compute the length of a side of a box.

In [ ]:
box_length = np.sqrt((xy[1][1] - xy[0][1])**2 + (xy[1][0] - xy[0][0])**2)

The interpixel distance is then 4 inches / `box_length`.  We will compute it in centimeters.

In [ ]:
interpixel_distance = 4 / box_length * 2.54

Now that we know the interpixel distance, we can burn a scale bar.  To do this, we simply set a stripe of pixels in a black area of the image to be white (or vice versa).  The upper left corner of the image is black and makes for a nice place to put a scale bar.  We will burn a scale bar of 5 cm.

In [ ]:
def burn_scale_bar(im, length, i_pos, j_pos, width=4, white=True):
    """
    Burn a horizontal scale bar starting at i_pos, j_pos.
    All units of inputs are interpixel distance.
    """
    im_out = im.copy()
    
    if white:
        pixel_val = skimage.dtype_limits(im)[1]
    else:
        pixel_val = skimage.dtype_limits(im)[0]
    
    im_out[i_pos-(width//2):i_pos+(width//2), j_pos:j_pos+length] = pixel_val
    return im_out
    
# Put black scale bar
im_sb = burn_scale_bar(im, 5 / interpixel_distance, 30, 10)

# Look at it
with sns.axes_style('white'):
    skimage.io.imshow(im_sb)

Note, though, that in the case of an image like this, we could just put something like, "Jelly fish boxes are 4$\times$4 inches," in our caption, and be done with it.  I'm showing you this because it is important to know how to do for many of the images you come across.

`scikit-image` currently does not have capability to add text.  You can make scale bars with text easily when preparing figures for publication within a Python framework using [OpenCV](http://opencv.org/) or by using [Fiji](http://fiji.sc/).

I want to make a final couple of notes on scale bars.  Some will argue that the scale bars could be added as vector graphics with nice, clean text.  There is no inherent problem with this, but in practice, images often get stretched and rearranged for display in presentations or in publications.  It is often difficult to control how various layers on objects will scale when this is done.  I prefer to keep myself out of trouble and burn in the scale bars.  Finally, make sure your scale bars are not burned in to the image you are actually analyzing, only those for viewing!  Remember, by burning in the scale bar in something you are analyzing, you are changing *data*!

## False coloring for image display

Remember, showing an image on screen or printing it on a piece of paper is *plotting data*.  We are free to choose *how* we plot it.  The common convention is to show the image in gray scale, but we need not do that.  We can instead look at the image in false color.  ("False" is a misnomer here, since any color is just a representation of photon counts on a detector.)  *This is not adding or subtracting information; it is merely changing the way we display it.*  Since images are typically sequential data, sequential colormaps are preferred.  In cases where we might have stark separation between dark and light, a diverging colormap is useful.  My favorite sequential and diverging colormaps for images are, respectively, `plt.cm.viridis` (Matplotlib's fantastic new default colormap) and `plt.cm.RdBu_r`.  Let's look at the image with these colormaps.

In [ ]:
# Set up figure with subplots
with sns.axes_style('white'):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    ax[0].imshow(im, cmap=plt.cm.viridis)
    ax[1].imshow(im, cmap=plt.cm.RdBu_r)

It is often easier to see fine detail with false coloring.  While not important for our analysis, we can, for example, see the nuances in brightness in the sand on which the jellyfish sit.

## Loading a stack of images
While we have done some nice things with single images, we really want to be able to sequentially look at all of the images of the movie, a.k.a. the stack of images.

`scikit-image` has a convenient way to do this using the `skimage.io.ImageCollection` class.  We simple give it a string matching the pattern of the file names we want to load.  In our case, this is `im_glob` that we have already defined.  An important kwarg is `conserve_memory`.  If there are many images, it is best to select this to be `True`.  For few images, selecting this to be `False` will result in better performance because all of the images will be read into RAM.  Let's load the daytime movie as an `ImageCollection`.  (Note: we're not actually loading them all in; we just creating an object that knows where the images are so we can access them at will.)

In [ ]:
ic = skimage.io.ImageCollection(im_glob, conserve_memory=True)

In [ ]:
# Line profiler (you might not have this installed)
%load_ext line_profiler

import glob
import os
import numpy as np
from PIL import Image
import skimage.io
import scipy.ndimage

# The directory containing daytime data
data_dir = '../data/abrams_et_al/Cassiopea_Pulsation/day'

# Glob string for images
im_glob = os.path.join(data_dir, '*.TIF')

# Get list of files in directory
im_list = glob.glob(im_glob)

# Load image collection
ic = skimage.io.ImageCollection(im_glob, conserve_memory=True)

def load_40_images(ic, im_list, method):
    ims = [None] * 40
    if method == 'collection':
        for i in range(40):
            ims[i] = ic[i][0,:,:]
    elif method == 'scipy':
        for i in range(40):
            ims[i] = scipy.ndimage.imread(im_list[i])
    elif method == 'PIL':
        for i in range(40):
            im_PIL = Image.open(im_list[i])
            ims[i] = np.array(im_PIL.getdata(), np.uint16)
    elif method == 'skimage.io':
        for i in range(40):
            ims[i] = skimage.io.imread(im_list[i])[0,:,:]
            
print('ImageCollection:')
%timeit load_40_images(ic, im_list, method='collection')

print('\nscipy:')
%timeit load_40_images(ic, im_list, method='scipy')

print('\nPIL:')
%timeit load_40_images(ic, im_list, method='PIL')

print('\nskimage.io:')
%timeit load_40_images(ic, im_list, method='skimage.io')

%lprun -T lp_results.txt -f gillespie_ssa gillespie_ssa(\
        params, simple_propensity, simple_update, population_0, time_points)

In [1]:
# Line profiler (you might not have this installed)
%load_ext line_profiler

import glob
import os
import numpy as np
import scipy.ndimage

# The directory containing daytime data
data_dir = '../data/abrams_et_al/Cassiopea_Pulsation/day'

# Glob string for images
im_glob = os.path.join(data_dir, '*.TIF')

# Get list of files in directory
im_list = glob.glob(im_glob)

def load_and_mean(im_list, n):
    im_means = np.empty(n)
    for i, fname in enumerate(im_list[:n]):
        im = scipy.ndimage.imread(fname)
        im_slice = im[300:500, 300:500]
        im_means[i] = im_slice.mean()
        
    return im_means

%lprun -T lp_results.txt -f load_and_mean load_and_mean(im_list, 400)


*** Profile printout saved to text file 'lp_results.txt'. 


In [ ]:
ims = [None] * 4000
for i in range(4000):
    if i % 100 == 0:
        print('i =', i, flush=True)
    ims[i] = scipy.ndimage.imread(im_list[i])

In [ ]:
skimage.io.imshow(ims[0])

Conveniently, the `ImageCollection` has nice properties.  Typing `len(ic)` tells how many images are in the collection.  `ic[157]` is the 157th image in the collection.  There is one problem, though.  All of the images are read in with all three channels.  We would like to only include one channel, since the others are redundant.  To do this, we can instantiate the `ImageCollection` with the `load_func` kwarg.  This specifies a function that reads in and returns a NumPy array with the image.  The default is `load_func=skimage.io.imread`, but we can write out own.  We'll write a function that loads the image and then just returns the red channel and reinstantiate the `ImageCollection` using that load function.

In [ ]:
def squish_rgb(fname):
    return skimage.io.imread(fname)[0,:,:]

ic = skimage.io.ImageCollection(im_glob, conserve_memory=True, 
                                load_func=squish_rgb)

We should also have time stamps for the data in the images.  We know the frame rate is 15 frames per second, so we can attach times to each image.  Normally, these would be in the metadata of the images and we would fish that out, but for this example, we will just generate our own time points (in units of seconds).

In [ ]:
fps = 15
t = np.arange(0, len(ic)) / fps

## Setting an ROI

A **region of interest**, or ROI, is part of an image or image stack that we would like to study, ignoring the rest.  Depending on the images we are analyzing, we may be able to automatically detect ROIs based on well-defined criteria.  Often, though, we need to manually pick regions in an image as our ROI.

`scikit-image` is an open source project that is constantly under development.  It currently does not have a way to specify ROIs, but it is on the list of functionality to be added.

So, I wrote my own ROI utility, which is included in [`bebi103_utils`](https://github.com/justinbois/bebi103_utils).  It takes a set of vertices that define a polygon, the inside of which constitutes a region of interest.  (Note that the polygon cannot have crossing lines.)  It returns a tuple that contains a **mask** for the ROI, a **bounding box**, and the mask for an image consisting entirely of the bounding box.  An ROI mask is the same shape as the image, except where the image has a pixel value, the ROI mask has a `True` value if the pixel is within the ROI and has a `False` value it if is outside.  The bounding box defines the smallest square subimage that completely contains the ROI.  Finally, it is convenient to have a mask that is reindexed such that (0,0) is the upper left corner of the bounding box.

To get the vertices, we can use the `plt.ginput()` function.  This cannot be demonstrated in an IPython notebook with inline `matplotlib`, so I do not show it here.  The syntax is below, with `ginput()` commented out, with its output instead hard coded.  In this example, I drew a square in the image to select a single box.

In [ ]:
# Select the condos as ROIs.  Use plt.ginput to get vertices
# Uncomment the below to use ginput.  Hit enter when finished selecting verts
# verts = plt.ginput(100)

verts = [(348.79397748093913, 246.73039544685778),
         (491.30157835613034, 249.91294732744518),
         (487.05817584868049, 392.77416507825723),
         (347.37950997845581, 386.76267819270322)]

Now we can use `bebi103.verts_to_roi()` to make make the ROI.

In [ ]:
roi, roi_bbox, roi_box = bebi103.verts_to_roi(verts, *ic[0].shape)

Now that we have defined the ROI, let's look at it.  We will use a trick where we take a grayscale image, convert it to RGB, and then add more blue in certain regions to highlight.

In [ ]:
# Make grayscale image that is now RGB
im = np.dstack(3*[skimage.img_as_float(ic[0])])

# Max out blue channel
im[roi,2] = skimage.dtype_limits(im)[1]

# Look at the image
with sns.axes_style('white'):
    skimage.io.imshow(im)

If you just want to look at the region of the image that bounds the ROI, you can do the following.

In [ ]:
# Get cropped image and ROI within it
im = ic[0][roi_bbox]
im_cropped_roi = roi_box

# Make grayscale image that is now RGB
im = np.dstack(3*[skimage.img_as_float(im)])

# Max out blue channel
im[im_cropped_roi,2] = skimage.dtype_limits(im)[1]

# Look at the image
with sns.axes_style('white'):
    skimage.io.imshow(im)

So, when working with ROIs, the `roi`, `roi_bbox`, and `roi_box` are useful tools to automatically index the original image to get what you are interested in.  We could also modify our `load_func` for our `ImageCollection` to just load in an ROI.

In [ ]:
# Define load_func that only gives region bounding ROI
def load_roi(fname, roi_bbox=None):
    if roi_bbox is None:
        return skimage.io.imread(fname)[0,:,:]
    else:
        return skimage.io.imread(fname)[0,:,:][roi_bbox]

# Load image collection
ic = skimage.io.ImageCollection(im_glob, conserve_memory=True, 
                                load_func=load_roi, roi_bbox=roi_bbox)

# Look at first image
with sns.axes_style('white'):
    skimage.io.imshow(ic[0])

## Some simple analysis

We are interested in the rate at which the jellyfish pulse.  Since the jellyfish are dark on a black background, we could just watch how the total pixel intensity of our respective ROIs change over time to get the pulsing frequency.  This will not really tell us about the shape of the jellyfish or any fine detail, but it will hopefully be enough to get us a rough estimate of pulsing frequency.

To be more concrete, our goal is to find the distribution of inter-contraction times.  If the distribution is tightly peaked, we have periodic pulsing and we can estimate a frequency.  Otherwise, we might notice pause events that we can see qualitatively in the movies.

We will analyze the jellyfish in our current ROI.  To start with, we'll just compute the total pixel intensity through time.

In [ ]:
# Set up NumPy array to store total pixel intensity
total_int = np.empty(len(t))

# Look through and compute total intensity in the ROI
for i, im in enumerate(ic):
    total_int[i] = ic[i][roi_box].sum()

Let's plot the result!

In [ ]:
plt.plot(t, total_int)
plt.xlabel('time (s)')
plt.ylabel('total intensity')
plt.margins(0.02)

Since the intensity units are arbitrary, we can let's subtract the mean and rescale the data so they go from -1 to 1.

In [ ]:
total_int -= total_int.mean()
total_int = 1 + 2 / (total_int.max() - total_int.min()) \
                * (total_int - total_int.max())

It will be easier to inspect these data by looking at a smaller time window.  We'll look at the 30 seconds.

In [ ]:
tinds = np.s_[:fps*30]
plt.plot(t[tinds], total_int[tinds])
plt.xlabel('time (s)')
plt.ylabel('normalized intensity')
plt.margins(0.02)

Better yet, we can just plot it with Bokeh.

In [ ]:
df = pd.DataFrame(data={'time (s)': t, 'normalized intensity': total_int})
p = bokeh.charts.Line(df, x='time (s)', y='normalized intensity', 
                      color='dodgerblue')
bokeh.io.show(p)

### The periodogram
This looks quite periodic, so let's have a look at the periodogram.

In [ ]:
# Determine frequencies
f = np.fft.fftfreq(len(t)) * fps

# Compute power spectral density
psd = np.abs(np.fft.fft(total_int))**2 / len(t)

# Make plot
plt.plot(f[f>0], psd[f>0])
plt.xlabel('freq (Hz)')
plt.ylabel('PSD')
plt.margins(0.02)

There is a very strong peak near 1 Hz, which is the typical pulse frequency.  We could have guessed this just by looking at the plots.  Nonetheless, there are other frequencies present, most strikingly at about 0.4 Hz.

### Identifying peaks
Now let's identify the peaks in the time trace.  Our strategy is as follows.

1. Indentify peak regions as between positive and negative crossings of zero.
2. Localize the maximum of the peak as the maximum between upward and downward crossings of zero.
3. Get sub-sampling accuracy for peak placement by fitting a quadratic to the maximum point and its nearest neighbors, like we did in [Tutorial 7a](t7a_time_series.html).

We first write a quick function to use the quadratic fitting method to get a local maximum.

In [ ]:
def local_min(x, y):
    """
    Fit three points to quadratic and return location of maximum or minimum.
    """
    a, b, c = np.polyfit(x, y, 2)
    x_max = -b / 2 / a
    return x_max, a*x_max**2 + b*x_max + c

Now, we'll find the positions of maxima to the accuracy of our sampling. 

In [ ]:
# Find up and down crossing indices
up_crossing_inds = np.where(
    np.logical_and(total_int[:-1] < 0, total_int[1:] >= 0))[0]
down_crossing_inds = np.where(
    np.logical_and(total_int[:-1] > 0, total_int[1:] <= 0))[0] + 1

# Make sure upcrossing are first
if down_crossing_inds[0] < up_crossing_inds[0]:
    down_crossing_inds = down_crossing_inds[1:]
    
# Make sure downcrossing last
if up_crossing_inds[-1] > down_crossing_inds[-1]:
    up_crossing_inds = up_crossing_inds[:-1]
    
# Find maxima
t_peaks = np.empty(len(up_crossing_inds))
peaks = np.empty_like(t_peaks)
for i, uc in enumerate(up_crossing_inds):
    ind = uc + np.argmax(total_int[uc:down_crossing_inds[i]+1])
    t_peaks[i], peaks[i] = local_min(t[ind-1:ind+2], total_int[ind-1:ind+2])

Let's see how we did my making a plot.

In [ ]:
# Set up the figure (this is like a canvas you will paint on)
p = bokeh.plotting.figure(background_fill='#DFDFE5', plot_width=650, 
                          plot_height=450)
p.xgrid.grid_line_color = 'white'
p.ygrid.grid_line_color = 'white'
p.xaxis.axis_label = 'time (s)'
p.yaxis.axis_label = 'normalized intensity'

# Specify the glyphs
p.line(t, total_int, color='dodgerblue')
p.circle(t_peaks, peaks, size=5, color='tomato')

bokeh.io.show(p)

Looks like we did a good job finding the peaks.  Now, let's compute the inter-contraction times and see how they change over the course of the reading.

In [ ]:
# Intercontractions times
t_int = np.diff(t_peaks)

# Plot them!
plt.plot(t_int)
plt.xlabel('contraction number')
plt.ylabel('intercontraction time (s)')
plt.margins(0.02)

We get occasional pauses in contractions with an average contraction rate of about 1.05 Hz, as we expected from the periodogram.  Apparently, the jellyfish starting contracting faster between contractions 120 and 150.  

### Distribution of inter-contraction times
Let's now compute and plot the ECDF of inter-contraction times.

In [ ]:
# Make x and y values for ECDF
x = np.sort(t_int)
y = np.arange(1, len(x)+1) / len(x)
plt.plot(x, y, '.')
plt.xlabel('intercontraction time (s)')
plt.ylabel('ECDF')

The tails of the ECDF are shallower than a Gaussian, which implies that there are pausing events and also faster contractions.  We can see this more concretely with a Q-Q plot.

In [ ]:
# Compute mean and standard deviation based on middle 50th percentile
mu = x.mean()
sigma = x.std()

# Make draws
theor_x = np.array(
    [np.sort(st.norm.rvs(mu, sigma, size=len(x))) for _ in range(1000)])

# Upper and lower bounds
low_theor, up_theor = np.percentile(theor_x, (2.5, 97.5), axis=0)

# Plot Q-Q plots with 95% conf.
plt.fill_between(x, up_theor, low_theor, alpha=0.7)

# Plot 45 degree line
x_lim = plt.gca().get_xlim()
plt.plot(x_lim, x_lim, 'k-')

# Tidy up
plt.margins(0.02)
plt.xlabel('time between contractions (s)')
plt.ylabel('time between contractions (s)')

The non-Gaussianity is more pronounced at the high end of the inter-contraction time, indicative of occasional pauses.

## Your turn

Can you do the similar analysis with the night time data set and compare the results to the day?  Can you do it for all eight jellyfish?  What do you observe?  This is actually one of your homework problems.